## DeepDAO and API requests

Here we learn how to make API requests and we refresh a bit of plotting.

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(pd.__version__)
print(np.__version__)
print(requests.__version__)
print(plt.matplotlib.__version__)


Import authentication key.

In [ ]:
# https://martin-thoma.com/configuration-files-in-python/

import json

with open("../.private/keys.json") as keys_file:
    KEYS = json.load(keys_file)

APIKEY = KEYS['DEEPDAO']
# Note: don't print the key, or if you do, delete the cell's output
# (cell outputs are saved and can be sent to Github).

DeepDao Endpoint

In [ ]:
ENDPOINT = 'https://api.deepdao.io/v0.1/'

Let's create a function to communicate with the REST endpoint. 

In [ ]:
def api(query, params=None, post=False):

    # https://datagy.io/python-requests-authentication/
    headers={
        'x-api-key': APIKEY,
        'accept': 'application/json'
    }

    if post:
        response = requests.post(ENDPOINT + query,
                                headers=headers,
                                json=params)
    else:
        response = requests.get(ENDPOINT + query,
                                headers=headers,
                                params=params)

    print(response)
    return response.json()


Let's try our first DeepDao query.

In [ ]:
query = "ecosystem"

res = api(query)

Any response code greater than 400 indicates an error, else it is a success. See:

https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

In [ ]:
 res


Get all DAOs in DeepDAO.

In [ ]:
query = "organizations"

res = api(query)

In [ ]:
res

Save the list for future reference.

In [ ]:
df = pd.DataFrame(res['data']['resources'])
df.to_json("data/daos_deepdao.json", orient="records")

## Visually explore the saved file.
# Is it a mess? Prettify it with VS Code auto-formatter. 
# Don't you remember the shorcut? Check our python_warmup lecture.

#### Example: fetch data, insert into data frame, clean it, and plot it.

Treasuries by chain.

In [ ]:
query = "ecosystem/chains_treasury_breakdown"

res = api(query)



In [ ]:
res

Convert the result into a data frame for easier plotting.

In [ ]:
df = pd.DataFrame()

for d in res['data']:
    df = df.append(res['data'][d], ignore_index=True)


df.info()

Shorten column names.

In [ ]:
df.columns = [
    'chainId', 'chainName', 'orgs', 'props', 'votes', 'treas', 'usd', 'holders'
]

df.info()


Format data.

In [ ]:
def cleanData(x):
    return 0 if x == 'N/A' else x.replace(',', '')
    
int_columns = ['orgs', 'props', 'votes', 'treas', 'usd', 'holders']
for col in int_columns:
    # print(col)
    # df[col] = df[col].apply(lambda x : x.replace(',', ''))
    df[col] = df[col].apply(cleanData)


In [ ]:
df['orgs'] = df['orgs'].astype('float')
df['props'] = df['props'].astype('float')
df['votes'] = df['votes'].astype('float')
df['treas'] = df['treas'].astype('float')
df['usd'] = df['usd'].astype('float')
df['holders'] = df['holders'].astype('float')



In [ ]:
df.info()

Plot it.

In [ ]:
import matplotlib.pyplot as plt

df['chain'] = df['chainName'].apply(lambda x : x[0:3])
df = df.set_index(df['chain'])


fig, axes = plt.subplots(nrows=2, ncols=4)

counter = 0
for c in int_columns:
    x = 0 if counter < 4 else 1 
    y = counter % 4
    # print(c, x, y)
    ax = df[["chain", c]].plot.bar(x="chain", y=c, ax=axes[x,y], figsize=(20,10))
    # ax.set_title(c)
    ax.set_xlabel('')
    counter += 1


More compact (but with less control).

In [ ]:
df[int_columns].plot.pie(subplots=True, figsize=(20,10))


#### A few more queries...

In [ ]:
query = "ecosystem/gov_platforms"
res = api(query)

In [ ]:
res

In [ ]:
query = "proposals/active_proposals"
res = api(query, { 'page': 1 })

In [ ]:
res

#### Time Series

In [ ]:
query = "timeseries/daily_proposals"
params = {
    'platforms': "11",
    'startDate': "2018-01-01",
    'endDate': "2022-09-01",
}
res = api(query, params)


In [ ]:
res

In [ ]:
df = pd.DataFrame(res)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')


In [ ]:
df.info()
# df.describe()

In [ ]:
df.plot.box()


In [ ]:
df.plot.area(figsize=(12, 4))


In [ ]:
df.plot.density()

In [ ]:
df.plot.hist(bins=100)

In [ ]:
df.plot.line()

## Badger DAO

In [ ]:
badgerDAO = "fe4cd05c-f6cb-461a-add9-bcdb1365fc0e"


In [ ]:
query = "timeseries/daily_dao_proposals/" + badgerDAO

params = { 
    'startDate': "2018-01-01",
    'endDate': "2022-09-01",
}

res = api(query, params)

In [ ]:
len(res)

In [ ]:
res

In [ ]:
df_badger = pd.DataFrame(res)
df_badger['date'] = pd.to_datetime(df_badger['date'])
df_badger = df_badger.set_index('date')
df_badger.info()

In [ ]:
df_badger.describe()

In [ ]:
df_badger[df_badger['counter'] == df_badger['counter'].max()]

In [ ]:
query = "timeseries/daily_dao_treasury/" + badgerDAO

params = { 
    'startDate': "2018-01-01",
    'endDate': "2022-09-01",
}

res = api(query, params)
len(res)

In [ ]:
res

In [ ]:
df_badger_treasury = pd.DataFrame(res['liquid'])
df_badger_treasury['date'] = pd.to_datetime(df_badger_treasury['date'])
df_badger_treasury = df_badger_treasury.set_index('date')
df_badger_treasury.info()

In [ ]:
df_badger_treasury.plot()

In [ ]:
df_badger = df_badger.join(df_badger_treasury, how="outer")

In [ ]:
df_badger.head()

In [ ]:
df_badger.info()

In [ ]:
df
df.plot(x=df.index, y=["counter", "aum"])